<a href="https://colab.research.google.com/github/Mehedi16009/Mali-GPU-Firmware-Attestation-Defense-system-Against-MOLE-Attack-/blob/main/Firmware_Attestation_MaliGPU_Prototype_(Defense_System_Against_MOLE_Attack).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔐 Firmware Attestation Prototype (Google Colab Demo)

This notebook demonstrates a **software-only prototype** of *Firmware Attestation* —  
a defense against attacks like **MOLE**, where malicious actors replace GPU MCU firmware.

I show how to:
1. **Set up environment**  
   Install Python libraries (`cryptography`) and C libraries (`OpenSSL`, `gcc`) in Colab.

2. **Create a dummy firmware binary**  
   A fake `firmware.bin` simulates MCU firmware.

3. **Generate cryptographic keys**  
   - Private key (`mcu_priv.pem`) — kept by the trusted vendor.  
   - Public key (`mcu_pub.pem`) — stored securely in TEE/ROM in real systems.  

4. **Sign firmware**  
   Use the private key to sign `firmware.bin` → produces `firmware.sig`.

5. **Verify firmware (C program)**  
   - Compile and run `verify` (C with OpenSSL).  
   - Checks `firmware.bin` + `firmware.sig` against `mcu_pub.pem`.  
   - If signature matches, firmware is authentic.  

6. **Simulate firmware loader**  
   - `loader.py` mimics Linux/TEE calling the verifier.  
   - Loads firmware if verified, rejects if not.  

7. **Attack simulation**  
   - Modify (tamper) `firmware.bin`.  
   - Show that the old signature no longer validates → firmware rejected.  

---

## 📌 Why this matters
- In MOLE, the attacker abuses unverified `/lib/firmware` loads.  
- With attestation, only **signed firmware** is accepted.  
- This prototype demonstrates the **core defense mechanism**.  

---

## ⚠️ Notes
- In this demo, the public key is just a file. In production it must be stored in **secure world hardware** (e.g., OP-TEE, TrustZone, TPM).  
- This Colab version **cannot emulate a real Mali GPU** or OP-TEE, but it fully shows the mechanism.  

---


In [1]:
# @title Step 1. Install required libraries
!apt-get update -qq
!apt-get install -y gcc libssl-dev
!pip install cryptography


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gcc is already the newest version (4:11.2.0-1ubuntu1).
gcc set to manually installed.
libssl-dev is already the newest version (3.0.2-0ubuntu1.19).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [2]:
# @title Step 2. Create a dummy firmware binary (256 bytes of random data)
!dd if=/dev/urandom of=firmware.bin bs=1 count=256
!ls -lh firmware.bin


256+0 records in
256+0 records out
256 bytes copied, 0.000695795 s, 368 kB/s
-rw-r--r-- 1 root root 256 Sep  6 12:15 firmware.bin


In [3]:
# @title step 3. Signing tool (This will generate keypairs and sign the firmware.) [sign_firmware.py]

from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import ec
import argparse, sys

def gen_keypair(priv_out="mcu_priv.pem", pub_out="mcu_pub.pem"):
    priv = ec.generate_private_key(ec.SECP256R1())
    priv_bytes = priv.private_bytes(
        serialization.Encoding.PEM,
        serialization.PrivateFormat.PKCS8,
        serialization.NoEncryption()
    )
    pub = priv.public_key().public_bytes(
        serialization.Encoding.PEM,
        serialization.PublicFormat.SubjectPublicKeyInfo
    )
    with open(priv_out, "wb") as f:
        f.write(priv_bytes)
    with open(pub_out, "wb") as f:
        f.write(pub)
    print(f"Generated keypair: {priv_out}, {pub_out}")

def sign_file(priv_path, infile, sig_out):
    with open(priv_path, "rb") as f:
        priv = serialization.load_pem_private_key(f.read(), password=None)
    with open(infile, "rb") as f:
        data = f.read()
    signature = priv.sign(data, ec.ECDSA(hashes.SHA256()))
    with open(sig_out, "wb") as f:
        f.write(signature)
    print(f"Signed {infile} -> {sig_out}")

# Demo run inside Colab
gen_keypair()
sign_file("mcu_priv.pem", "firmware.bin", "firmware.sig")


Generated keypair: mcu_priv.pem, mcu_pub.pem
Signed firmware.bin -> firmware.sig


In [18]:
# @title Step 4. Verification program (C)[Verification program (C)]
c_code = """
#include <stdio.h>
#include <stdlib.h>
#include <openssl/evp.h>
#include <openssl/pem.h>

int verify_ecdsa(const char *pubkey_path, const char *firmware_path, const char *sig_path) {
    FILE *fpub = fopen(pubkey_path, "r");
    if (!fpub) { perror("fopen pubkey"); return -1; }
    EVP_PKEY *pkey = PEM_read_PUBKEY(fpub, NULL, NULL, NULL);
    fclose(fpub);
    if (!pkey) { fprintf(stderr, "Failed to read public key\\n"); return -2; }

    FILE *f = fopen(firmware_path, "rb");
    fseek(f, 0, SEEK_END); long flen = ftell(f); fseek(f, 0, SEEK_SET);
    unsigned char *data = malloc(flen); fread(data, 1, flen, f); fclose(f);

    FILE *fs = fopen(sig_path,"rb");
    fseek(fs, 0, SEEK_END); long slen = ftell(fs); fseek(fs, 0, SEEK_SET);
    unsigned char *sig = malloc(slen); fread(sig, 1, slen, fs); fclose(fs);

    EVP_MD_CTX *mdctx = EVP_MD_CTX_new();
    int ok = 0;
    if (EVP_DigestVerifyInit(mdctx, NULL, EVP_sha256(), NULL, pkey) == 1) {
        if (EVP_DigestVerify(mdctx, sig, slen, data, flen) == 1) {
            ok = 1;
            printf("Verification: OK\\n");
        } else {
            printf("Verification: FAILED\\n");
        }
    }
    EVP_MD_CTX_free(mdctx);
    EVP_PKEY_free(pkey);
    free(data); free(sig);
    return ok ? 0 : 1;
}

int main(int argc, char **argv) {
    if (argc != 4) {
        fprintf(stderr, "Usage: %s pub.pem firmware.bin firmware.sig\\n", argv[0]);
        return 2;
    }
    return verify_ecdsa(argv[1], argv[2], argv[3]);
}
"""

with open("verify_firmware.c", "w") as f:
    f.write(c_code)

# Compile verifier
!gcc verify_firmware.c -o verify -I/usr/include/openssl -lssl -lcrypto

In [22]:
# @title Step 5. Loader simulation (Python) [This mimics the kernel/driver calling verification before “loading” the firmware.]
import subprocess

def load_firmware(pubkey, firmware, signature):
    cmd = ["/content/verify", pubkey, firmware, signature] # Use the absolute path to the executable
    print("Running verifier:", " ".join(cmd))
    r = subprocess.run(cmd, capture_output=True, text=True)
    print(r.stdout.strip())
    if r.returncode == 0:
        print("[loader] Firmware verified — proceed to load firmware (simulated).")
    else:
        print("[loader] Firmware verification FAILED — reject firmware.")
    return r.returncode

# Test with signed firmware
load_firmware("mcu_pub.pem", "firmware.bin", "firmware.sig")

Running verifier: /content/verify mcu_pub.pem firmware.bin firmware.sig
Verification: FAILED
[loader] Firmware verification FAILED — reject firmware.


1

In [15]:
# @title Step 6. Tampering test (Tamper with firmware) [Now I simulate an attacker modifying firmware after it’s signed.]
!echo "MALICIOUS PATCH" >> firmware.bin

In [23]:
# @title Step 7. Run loader again: [Expected result: FAILED/ Rejected firmware]
load_firmware("mcu_pub.pem", "firmware.bin", "firmware.sig")


Running verifier: /content/verify mcu_pub.pem firmware.bin firmware.sig
Verification: FAILED
[loader] Firmware verification FAILED — reject firmware.


1

In [24]:
load_firmware("mcu_pub.pem", "firmware.bin", "firmware.sig")

Running verifier: /content/verify mcu_pub.pem firmware.bin firmware.sig
Verification: FAILED
[loader] Firmware verification FAILED — reject firmware.


1

In [25]:
load_firmware("mcu_pub.pem", "firmware.bin", "firmware.sig")

Running verifier: /content/verify mcu_pub.pem firmware.bin firmware.sig
Verification: FAILED
[loader] Firmware verification FAILED — reject firmware.


1

In [20]:
!ls -l

total 44
-rw-r--r-- 1 root root   272 Sep  6 12:23 firmware.bin
-rw-r--r-- 1 root root    72 Sep  6 12:15 firmware.sig
-rw-r--r-- 1 root root   241 Sep  6 12:15 mcu_priv.pem
-rw-r--r-- 1 root root   178 Sep  6 12:15 mcu_pub.pem
drwxr-xr-x 1 root root  4096 Sep  3 13:36 sample_data
-rwxr-xr-x 1 root root 16848 Sep  6 12:24 verify
-rw-r--r-- 1 root root  1483 Sep  6 12:24 verify_firmware.c
